<a href="https://colab.research.google.com/github/Ezendos/paszi/blob/master/Algorithms.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

# Тема 5
## [Ссылка на блокнот с заданием](https://github.com/Ezendos/paszi/Tasks2.ipyndb)

# Модуль functools
Предоставляемые модулем functools инструменты позволяют адаптировать или расширять функции и другие вызываемые объекты без полного их переписывания.

Для начала, вспомним, что такое декоратор. В своей основе декораторы Python позволяют расширять и изменять поведение вызываемых объектов (функций, методов и классов) без необратимой модификации самих вызываемых объектов.

[Блокнот Дмитрия Юрьевича по декораторам](https://github.com/dm-fedorov/advanced-python/blob/ee9316f1779b1f539084efd1c2c9473cb190c0a1/about_decorator.ipynb)

[Cтатья по декораторам на realpython.com](https://realpython.com/primer-on-python-decorators)

Ниже приведен пример простого декоратора функций.

In [1]:
def do_twice(func):     # декоратор это функция,
    def wrapper():      # которая возвращает некую внутренную оберточную функцию
        func()
        func()
    return wrapper

def say_hello():
    print("hello!")
    
say_hello = do_twice(say_hello) # декорирование - это, по сути, замена оригинальной функции на модифицированную

say_hello()

hello!
hello!


Такая форма записи `say_hello = my_decorator(say_hello)` несколько громоздка и не способствует читаемости кода. Python позволяет использовать символ `@`, чтобы применить к функции декоратор.

In [2]:
@do_twice             # функция декорируется при определении,
def say_goodbye():    # поэтому получить доступ к оригиналу становится проблематично
    print("goodbye!")
    
say_goodbye()

goodbye!
goodbye!


Применяя декоратор, мы подменяем одну функцию другой, из-за чего скрываются метаданные, закрепленные за оригинальной функцией.

Попробуем узнать имя декорированной функии:

In [3]:
say_goodbye.__name__

'wrapper'

Получили имя обёрточной функции. Такие вещи делают отладку и работу с интерпретатором Python неуклюжей и трудоемкой. К счастью, существует быстрое решение этой проблемы...

### `@functools.wraps`

Это декоратор, который можно использовать на оберточную функцию наших декораторов для того, чтобы копировать потерянные метаданные из оригинальной функции.

Пример:

In [4]:
import functools

In [5]:
def do_twice(func):
    @functools.wraps(func)   # добавим к оберточной функции functools.wraps
    def wrapper():
        func()
        func()
    return wrapper

@do_twice
def say_goodbye():
    print("goodbye!")

In [6]:
say_goodbye.__name__

'say_goodbye'

Посмотрим, как он это делает

Определение из functools.py :

```python
def wraps(wrapped,
          assigned = WRAPPER_ASSIGNMENTS,
          updated = WRAPPER_UPDATES):
    """Decorator factory to apply update_wrapper() to a wrapper function

       Returns a decorator that invokes update_wrapper() with the decorated
       function as the wrapper argument and the arguments to wraps() as the
       remaining arguments. Default arguments are as for update_wrapper().
       This is a convenience function to simplify applying partial() to
       update_wrapper().
    """
    return partial(update_wrapper, wrapped=wrapped,
                   assigned=assigned, updated=updated)
```

Описание говорит нам, что `wraps` это лишь удобный способ применять некий `partial()` к `update_wrapper()`. Разберёмся по порядку.

### `functools.partial(func, /, *args, **keywords)`

Возвращает partial-объект (вызываемый объект), с заранее заданными аргументами.

Пример:

In [7]:
from functools import partial

def power(x,y):
    return pow(x,y)

# создадим новую функцию, которая возводит во вторую степень
power2 = partial(power, y=2)

power2(5)

25

В данном примере partial-объект похож на функцию, но есть некоторые отличия. Например атрибуты `__name__` и `__doc__` не создаются по умолчанию:

In [8]:
power2.__name__ # выдаст ошибку

AttributeError: 'functools.partial' object has no attribute '__name__'

In [9]:
power2.__name__ = 'pow2' # имя можно задать
power2.__name__

'pow2'

partial-объекты позволяют работать c любыми вызываемыми объектами, в том числе и экземплярами классов. В основном, применяются для повторного использования кода в более сложных программах.

### `functools.update_wrapper(wrapper, wrapped, assigned=WRAPPER_ASSIGNMENTS, updated=WRAPPER_UPDATES)` 
Функция `update_wrapper()` позволяет добавлять и копировать атрибуты из исходной функции в функцию-обертку. Значения по умолчанию атрибутов, добавляемых в функцию-обертку, определяются c помощью константы `WRAPPER_ASSIGNMENTS`, тогда как значения по умолчанию, обновляющие соответствующие атрибуты функции-обертки, - c помощью константы `WRAPPER_UPDATES`.

---
### `functools.reduce(function, iterable[, initializer])` - Кумулятивное вычисление

Функция `reduce()` получает вызываемый объект и последовательность данных в качестве аргументов. Результатом ее работы является единственное значение, основанное на вызове объекта со значениями из последовательности и накоплении результатов. Если задан `initializer`, он помещается в начале последовательности.

In [10]:
functools.reduce(lambda a, b: a+b, [1, 2, 3, 4, 5], 10)

25

Пример эквивалентен (((((10+1)+2)+3)+4)+5).

Другой пример, поиск максимального элемента в списке.

In [11]:
functools.reduce(lambda a,b : a if a > b else b, [1, 7, 3, 10, 5])

10

Стоит отметить, что `reduce()` очень практично использовать вместе с функциями из модуля `operator`

In [12]:
import operator
functools.reduce(operator.add, [1, 2, 3, 4, 5], 10)

25

---
### `@functools.lru_cache(maxsize=128, typed=False)` - мемоизация
Декоратор `@lru_cache()` обертывает функцию кешем LRU (least recently used - неиспользованный дольше всех). Аргументы функции используются для создания хеш-значения, которое сопоставляется c результатом. Последующие вызовы функции c теми же аргументами будут заменяться извлечением соответствующего значения из кеша. Кроме того, этот декоратор
добавляет в функцию методы, обеспечивающие проверку состояния `cache_info()` и очистку `cache_clear()` кеша. Оригинальная функция доступна через атрибут `__wrapped__`

Создание такого кеша также называют мемоизацией.

Если параметр `maxsize` равен значению `None`, LRU функционал отключен, и кеш растет безгранично.

In [13]:
# пример: числа фиббоначи
@functools.lru_cache(maxsize=None)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

[fib(n) for n in range(20)] # делаем несколько вызовов функции

fib.cache_info() # смотрим кеш

CacheInfo(hits=36, misses=20, maxsize=None, currsize=20)

Мы можем использовать информацию, возвращаемую `cache_info()`, чтобы понять, как работает кэш, и настроить его, чтобы найти подходящий баланс между скоростью работы и объемом памяти:

- `hits=36` – количество вызовов, которые  `@lru_cache` вернул непосредственно из памяти, поскольку они присутствовали в кэше;
- `misses=20` – количество вызовов, которые взяты не из памяти, а были вычислены;
- `maxsize=None` – это размер кэша, который мы определили, передав его  декоратору;
- `currsize=20` – текущий размер кэша.


`@functools.cache(user_function)` - то же самое, но размер не ограничен по-умолчанию.

---
# Модуль itertools
Модуль itertools включает ряд распространённых шаблонов итераторов, предназначенных для построения быстрого и эффективного в отношении памяти кода. Инструменты модуля основаны на конструкциях из языков программирования APL, Haskell и SML. Используются итераторы для обработки крупных файлов и потоков данных, для доступа к содержимому объектов без раскрытия их полного внутреннего представления.

[Блокнот Дмитрия Юрьевича по итераторам](https://github.com/dm-fedorov/advanced-python/blob/ee9316f1779b1f539084efd1c2c9473cb190c0a1/about_iterator.ipynb)


In [14]:
import itertools

---

### `itertools.count(start=0, step=1)` - Бесконечный счётчик

Функция `itertools.count(start=0, step=1)` создаёт бесконечный итератор. Можно задать начальное значение и шаг итерирования.

In [15]:
cnt = itertools.count(start=2020, step=4)
print(next(cnt))
print(next(cnt))
print(next(cnt))

2020
2024
2028


Пример использования функции вместе с `zip()`:

In [16]:
days = [366]*4  # создаем список вида [366, 366, 366, 366]
list(zip(itertools.count(2020, 4), days))

[(2020, 366), (2024, 366), (2028, 366), (2032, 366)]

---
### `itertools.accumulate(iterable[, func, *, initial=None])` - Итератор с нарастающим итогом
Принцип схож с вышеупомянутым `functools.reduce()`. Разница лишь в том, что accumulate возвращает итератор.

In [17]:
list(itertools.accumulate(range(1, 10)))

[1, 3, 6, 10, 15, 21, 28, 36, 45]

Записывем все вызовы данного итератора с помощью `list()`.

По умолчанию к элементам применяется `operator.add`. Можно, например, указать оператор умножения:

In [18]:
import operator
list(itertools.accumulate(range(1, 10), operator.mul))

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880]

---
### `itertools.cycle(iterable)` - Бесконечный итератор последовательности

С помощью `itertools.cycle()` создаётся кольцевой итератор. Прийдя к последнему значению, он вновь начинает с первого:

In [19]:
waltz = itertools.cycle(['un', 'deux', 'trois'])
print(next(waltz))
print(next(waltz))
print(next(waltz))
print(next(waltz))

un
deux
trois
un


---
### `itertools.repeat(object[, times])` - Бесконечный итератор одного объекта
Эта функция создает итератор, возвращающий одно и то же значение при каждом обращении к нему. Количество повторений можно ограничить параметром `times`.

In [20]:
s = "Sample Text"
rep = itertools.repeat(s, times=2)
print(next(rep))
print(next(rep))
print(next(rep))

Sample Text
Sample Text


StopIteration: 

Классический пример использования `itertools.repeat()` – итератор для функции `map()`:

In [21]:
nums = range(10)
squares = map(pow, nums, itertools.repeat(2))
list(squares)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

`map()` возвращает итератор, который применяет функцию к элементам последовательности.

---

### `itertools.starmap(function, iterable)` - map со звёздочкой

Раз мы заговорили о `map()`, полезно рассказать и о `itertools.starmap()`. Этот метод принимает функцию и список кортежей аргументов. Как если бы использовался оператор `*`:

In [22]:
squares = itertools.starmap(pow, [(0, 2), (1, 2), (2, 2)])
list(squares)

[0, 1, 4]

---
### `itertools.chain(*iterables)` - Цепочки итераторов

Иногда необходимо использовать нескольков итераторов. И независимо, и цепочкой один за другим. Для объединения итераторов используется `itertools.chain(*iterables)`.

Например, мы хотим использовать для отрисовки игральных карт независимые итераторы обозначений в углу поля карты:

In [23]:
num_cards = [str(i) for i in range(2, 11)]
face_cards  = ['J', 'Q', 'K', 'A']

list(itertools.chain(num_cards, face_cards))

['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

С помощью `itertools.chain()` также можно добавлять отдельные элементы в начало итератора:

In [24]:
def prepend(value, iterator):
    return itertools.chain([value], iterator)

list(prepend(1, [2, 3, 4]))

[1, 2, 3, 4]

---
### `itertools.islice(iterable, start, stop[, step])` - Итератор среза

Срез – удобный инструмент списков. Функция `itertools.islice()` позволяет переходить по любым объектам в формате среза.

---
### `itertools.compress(data, selectors)` - Фильтрация группы элементов

Функция `compress()` оставляет из итерируемых данных только те, что соответствуют позициям булевых селекторов:

In [25]:
numbers = [0, 1, 2, 3, 2, 1, 0]
selectors = [True, True, False, True]

list(itertools.compress(numbers, selectors))

[0, 1, 3]

---
## Комбинаторика

Модуль `itertools` также позволяет решать комбинаторные задачи.

### `itertools.combinations(iterable, r)` - Сочетания

Сочетания выбранные из множества `n` объектов комбинации  `m` объектов, отличающиеся хотя бы одним объектом. Порядок элементов не важен.

Например, мы хотим составить трёхцветный флаг из лент цветных тканей. Есть четыре цвета лент. Все варианты выбора тканей без учёта их расположения:

In [26]:
colors = ['white', 'yellow', 'blue', 'red']
for item in itertools.combinations(colors, 3):
    print(item)

('white', 'yellow', 'blue')
('white', 'yellow', 'red')
('white', 'blue', 'red')
('yellow', 'blue', 'red')


Порядок следования не имеет значения, поэтому все тройки цветов уникальны.

---
### `itertools.permutations(iterable, r=None)` - Перестановки

Те же сочетания, для которых важен порядок следования элементов. В продолжение предыдущего примера определим все варианты как мы можем составить флаг с учётом порядка следования цветов:

In [27]:
for item in itertools.permutations(colors, 3):
    print(item)

('white', 'yellow', 'blue')
('white', 'yellow', 'red')
('white', 'blue', 'yellow')
('white', 'blue', 'red')
('white', 'red', 'yellow')
('white', 'red', 'blue')
('yellow', 'white', 'blue')
('yellow', 'white', 'red')
('yellow', 'blue', 'white')
('yellow', 'blue', 'red')
('yellow', 'red', 'white')
('yellow', 'red', 'blue')
('blue', 'white', 'yellow')
('blue', 'white', 'red')
('blue', 'yellow', 'white')
('blue', 'yellow', 'red')
('blue', 'red', 'white')
('blue', 'red', 'yellow')
('red', 'white', 'yellow')
('red', 'white', 'blue')
('red', 'yellow', 'white')
('red', 'yellow', 'blue')
('red', 'blue', 'white')
('red', 'blue', 'yellow')


---

### `itertools.product(*iterables, repeat=1)` - Размещение с повторениями

Размещение с повторениями (выборка с возвращением) – это комбинаторное размещение объектов, в котором каждый объект может участвовать в размещении несколько раз.

Например, есть пин-код из четырех цифр. На каждой позиции стоит цифра от `0` до `9`. Позиции не зависят друг от друга. Переберем все возможные коды:

In [28]:
digits = range(10)
pincode_vars = itertools.product(digits, repeat=4)
for var in pincode_vars:
    print(var)

(0, 0, 0, 0)
(0, 0, 0, 1)
(0, 0, 0, 2)
(0, 0, 0, 3)
(0, 0, 0, 4)
(0, 0, 0, 5)
(0, 0, 0, 6)
(0, 0, 0, 7)
(0, 0, 0, 8)
(0, 0, 0, 9)
(0, 0, 1, 0)
(0, 0, 1, 1)
(0, 0, 1, 2)
(0, 0, 1, 3)
(0, 0, 1, 4)
(0, 0, 1, 5)
(0, 0, 1, 6)
(0, 0, 1, 7)
(0, 0, 1, 8)
(0, 0, 1, 9)
(0, 0, 2, 0)
(0, 0, 2, 1)
(0, 0, 2, 2)
(0, 0, 2, 3)
(0, 0, 2, 4)
(0, 0, 2, 5)
(0, 0, 2, 6)
(0, 0, 2, 7)
(0, 0, 2, 8)
(0, 0, 2, 9)
(0, 0, 3, 0)
(0, 0, 3, 1)
(0, 0, 3, 2)
(0, 0, 3, 3)
(0, 0, 3, 4)
(0, 0, 3, 5)
(0, 0, 3, 6)
(0, 0, 3, 7)
(0, 0, 3, 8)
(0, 0, 3, 9)
(0, 0, 4, 0)
(0, 0, 4, 1)
(0, 0, 4, 2)
(0, 0, 4, 3)
(0, 0, 4, 4)
(0, 0, 4, 5)
(0, 0, 4, 6)
(0, 0, 4, 7)
(0, 0, 4, 8)
(0, 0, 4, 9)
(0, 0, 5, 0)
(0, 0, 5, 1)
(0, 0, 5, 2)
(0, 0, 5, 3)
(0, 0, 5, 4)
(0, 0, 5, 5)
(0, 0, 5, 6)
(0, 0, 5, 7)
(0, 0, 5, 8)
(0, 0, 5, 9)
(0, 0, 6, 0)
(0, 0, 6, 1)
(0, 0, 6, 2)
(0, 0, 6, 3)
(0, 0, 6, 4)
(0, 0, 6, 5)
(0, 0, 6, 6)
(0, 0, 6, 7)
(0, 0, 6, 8)
(0, 0, 6, 9)
(0, 0, 7, 0)
(0, 0, 7, 1)
(0, 0, 7, 2)
(0, 0, 7, 3)
(0, 0, 7, 4)
(0, 0, 7, 5)
(0, 0, 7, 6)

---
### `itertools.combinations_with_replacement(iterable, r)` - Размещение

Рассмотрим также случай обычного размещения, когда  элементы могут повторяться, но каждое сочетание встречается только один раз:

In [29]:
letters = 'ABCD'

code_vars = itertools.combinations_with_replacement(letters, 2)
for var in code_vars:
    print(var)

('A', 'A')
('A', 'B')
('A', 'C')
('A', 'D')
('B', 'B')
('B', 'C')
('B', 'D')
('C', 'C')
('C', 'D')
('D', 'D')


# Модуль operator 

Время от времени в процессе программирования c использованием итераторов возникает необходимость в создании небольших функций для вычисления простых выражений. Иногда это можно реализовать c помощью лямбда-функций, но для некоторых операций новые функции вообще не нужны. Модуль `operator` содержит функции, которые соответствуют встроенным арифметическим операторам, операторам сравнения и другим стандартным операторам.

Многие имена функций используются для специальных методов без двойного подчеркивания. Для обратной совместимости многие из них имеют вариант с сохранением двойного подчеркивания. 

### Логические операции

Модуль `operator` предоставляет функции для определения булевых эквивалентов значений, отрицания значений для получения противоположного булевого значения, а также сравнения объектов c целью проверки их идентичности. 

In [30]:
import operator
operator.not_("")

True

In [31]:
operator.truth("3")

True

In [32]:
operator.truth("")

False

In [33]:
operator.is_(1, -5)

False

In [34]:
operator.is_not(1, -5)

True

### Операторы сравнения

Поддерживаются все операторы расширенного сравнения.

In [35]:
a = 1
b = 4
operator.lt(a, b)

True

In [36]:
operator.le(a, b)

True

In [37]:
operator.eq(a, b)

False

In [38]:
operator.ne(a, b)

True

In [39]:
operator.ge(a, b)

False

In [40]:
operator.gt(a, b)

False

### Арифметические операторы

Поддерживаются также арифметические операторы для манипулирования
числовыми значениями.

In [41]:
a = 1
b = -4
operator.abs(a)

1

In [42]:
operator.neg(a)

-1

In [43]:
operator.pos(b)

-4

In [44]:
operator.add(a, b)

-3

In [45]:
operator.floordiv(a, b)

-1

In [46]:
operator.mod(a, b)

-3

In [47]:
operator.mul(a, b)

-4

In [48]:
operator.pow(a, b)

1.0

In [49]:
operator.sub(a, b)

5

In [50]:
operator.truediv(a, b)

-0.25

In [51]:
operator.and_(a, b)

0

In [52]:
operator.invert(a)

-2

In [53]:
operator.lshift(b, a)

-8

In [54]:
operator.or_(a, b)

-3

In [55]:
operator.xor(a, b)

-3

### Операторы для работы c последовательностями

Операторы для работы c последовательностями можно разбить на четыре
группы: создание последовательностей, поиск элементов, доступ к содержимому
и удаление элементов последовательности.

Cоздание последовательностей:

In [56]:
a = [1, 2, 3]
b = ['a', 'b', 'c']
operator.concat(a, b)

[1, 2, 3, 'a', 'b', 'c']

Поиск элементов:

In [57]:
operator.contains(b, 'c')

True

In [58]:
operator.countOf(b, 'c')

1

In [59]:
operator.indexOf(a, 3)

2

Доступ к содержимому:

In [60]:
operator.getitem(b, 0)

'a'

In [61]:
operator.setitem(a, 1, 'z')
print(a)

[1, 'z', 3]


Удаление элементов последовательности:

In [62]:
operator.delitem(a, 1)
print(a)

[1, 3]


### Операторы, изменяющие операнды

В дополнение к стандартным операторам многие типы объектов поддерживают также операции, выполняемые “на месте”, т.е. c изменением самих операндов,
как операция +=. Эквивалентные функции предусмотрены и для таких операторов.

In [63]:
a = -3
b = 6
operator.iadd(a, b)

3

In [64]:
c = ['a', 'b', 'c']
d = ['z', 'x', 'v']
operator.iconcat(c, d)

['a', 'b', 'c', 'z', 'x', 'v']

# Модуль contextlib

Модуль contextlib содержит вспомогательные функции для работы c менеджерами контекста и инструкцией with.

### API менеджера контекста

Менеджер контекста отвечает за использование ресурсов в пределах блока
кода, возможно, c созданием менеджера контекста при входе в блок и его последующим уничтожением c очисткой ресурсов при выходе из блока. Например, файлы поддерживают API менеджера контекста, гарантирующего закрытие файла по
завершении чтения или записи данных.

In [65]:
with open('./pymotw.txt', 'w') as f:
    f.write('contents go here')

Контекстный менеджер активизируется инструкцией with, а соответствующий API включает два метода. Метод __enter__ () выполняется при входе в блок кода инструкции with. Он возвращает объект, который будет использоваться в пределах данного контекста. Когда поток управления покидает блок with, вызывается метод __exit__ () менеджера контекста, освобождающий использованные ресурсы.

In [66]:
class Context: 
    def __init__(self) :
        print ('__init__() ')
    def __enter__(self):
        print ('__enter__() ')
        return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        print ('__exit__() ')
with Context():
    print('Работа в контектсе')


__init__() 
__enter__() 
Работа в контектсе
__exit__() 


Метод __enter__() может вернуть любой объект, который будет связан c именем, указанным в предложении as инструкции with. В следующем примере метод Context () возвращает объект, который использует открытый контекст.

In [67]:
class WithinContext:
    def __init__(self, context):
        print('WithinContext.__init__ ({})'.format(context))
    def do_something(self):
        print('WithinContext.do_something()')
    def __del__(self) :
        print('WithinContext.__del__')
class Context:
    def __init__ (self):
        print('Context.__init__ ()')
    def __enter__(self):
        print ('Context.__enter__ () ')
        return WithinContext(self)
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('Context.__exit__()')
with Context() as c:
    c.do_something()

Context.__init__ ()
Context.__enter__ () 
WithinContext.__init__ (<__main__.Context object at 0x000001ED570328E0>)
WithinContext.do_something()
Context.__exit__()


### Менеджеры контекста как декораторы функций

Класс ContextDecorator добавляет в классы контекстных менеджеров поддержку, позволяющую использовать их не только в качестве менеджеров контекста, но и в качестве декораторов функций.

In [68]:
import contextlib

class Context(contextlib.ContextDecorator):
    
    def __init__(self, how_used):
        self.how_used = how_used
        print('__init__({})'.format(how_used))
        
    def __enter__(self):
        print('__enter__({})'.format(self.how_used))
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('__exit__({})'.format(self.how_used))

with Context('as context manager'):
    print('Doing work in the context')

print()
@Context('as decorator')
def func(message):
    print(message)
func('Doing work in the wrapped function')

__init__(as context manager)
__enter__(as context manager)
Doing work in the context
__exit__(as context manager)

__init__(as decorator)
__enter__(as decorator)
Doing work in the wrapped function
__exit__(as decorator)


### Закрытие открытых дескрипторов

Класс file обеспечивает непосредственную поддержку АРI менеджера контекста, однако в случае других объектов, представляющих открытые дескрипторы, это не гак. В разделе документации стандартной библиотеки, посвященном модулю contextlib, в качестве примера представлен объект, возвращаемый вызовом urllib.urlopen (). Некоторые другие устаревшие классы используют метод
close (), но не поддерживаютАР1 менеджера контекста. Гарантированное закрытие дескриптора обеспечивается созданием для него менеджера контекста c помощью функции closing ().

In [69]:
import contextlib

class Door:
    
    def __init__(self):
        print('__init__()')
        self.status = 'open'
    
    def close(self):
        print('close()')
        self.status = 'closed'

print('Normal Example:')
with contextlib.closing(Door()) as door:
    print('inside with statement: {}'.format(door.status))
print('outside with statement: {}'.format(door.status))

print('\nError handling example:')
try:
    with contextlib.closing(Door()) as door:
        print('raising from inside with statement')
        raise RuntimeError('error message')
except Exception as err:
    print('Had an error:', err)

Normal Example:
__init__()
inside with statement: open
close()
outside with statement: closed

Error handling example:
__init__()
raising from inside with statement
close()
Had an error: error message


### Игнорирование исключений

Во многих случаях исключения, возбуждаемые библиотеками, удобно игноировать, если ошибка указывает на достижение определенного состояния или может быть проигнорирована по другим причинам. Наиболее распространеным способом игнорирования исключений является использование инструкции try:except, содержащей в блоке except только инструкцию pass.

In [70]:
import contextlib

class NonFatalError(Exception):
    pass
def non_idempotent_operation():
    raise NonFatalError('The operation failed because of existing state')
    
try:
    print('trying non-idempotent operation')
    non_idempotent_operation()
    print('succeeded!')
except NonFatalError:
    pass
print ('done')


trying non-idempotent operation
done


Форму try:except можно заменить формой contextlib.suppress () для болee явного подавления класса исключений, возникающих в пределах блока with.

In [71]:
import contextlib

class NonFatalError(Exception):
    pass

def non_idempotent_operation():
    raise NonFatalError(
        'The operation failed because of existing state'
    )
    
with contextlib.suppress(NonFatalError):
    print('trying non-idempotent operation')
    non_idempotent_operation()
    print('succeeded!')
    
print('done')

trying non-idempotent operation
done


## [Ссылка на блокнот с заданием](https://github.com/Ezendos/paszi/Tasks2.ipyndb)